This notebook aims to generate Trauma Pacient's IDs for the MIMICIII trauma population.

**Notes**:
This notebook assumes you have access to MIMIC III (v1.4) on Google BigQuery. To get access follow the [link](https://physionet.org/content/mimiciii/1.4/)

To access MIMIC IV data in Google Colab, check the [link](https://colab.research.google.com/drive/1REu-ofzNzqsTT1cxLHIegPB0nGmwKaM0?usp=sharing#scrollTo=s-MoFA6NkkbZ)

# Set Up

In [1]:
# Import libraries
import os
import re   # for filter file

import gzip # for load .csv.gz file
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [2]:
%cd /content/drive/MyDrive/Yin_AutoEncoder/mimiciii_code/Project
import mimic_utils as utils
import mimiciii_icu_trauma_patient_cohort as trauma_cohort

/content/drive/.shortcut-targets-by-id/1z57lUhb4GSkXb08DWlFjx8kZ4Dg6Umsv/Yin_AutoEncoder/mimiciii_code/Project


In [3]:
# Project Path
PROJECT_PATH = '/content/drive/MyDrive/Yin_AutoEncoder/mimiciii_code/Project'

# Data Path
# mimic original dataset path
DATASET_PATH = os.path.join(PROJECT_PATH, '1.4')
PREPROCESS_DATA_PATH = os.path.join(PROJECT_PATH, 'preprocess')

# environment variables of BigQuery
PROJECT_ID = 'sepsis-mimic3'

In [4]:
# testing for big quene
# query = """
#       SELECT *
#       FROM `physionet-data.mimiciii_clinical.admissions`
#       LIMIT 5
#       """
# utils.run_query(query, PROJECT_ID)

# ICD9 CODE Selection



In [ ]:
# Load HMC's list of traumatic injury E-codes.
ICD9Code_file_path = os.path.join(PROJECT_PATH, 'Supplementary','ICD_Nonpoisoning_Cause_Matrix.xlsx')
ICD9Code_file_path
df_hmc_e = pd.read_excel(ICD9Code_file_path, sheet_name="Ecodes ICD 9")
# Reformat the codes to be consistent with MIMIC's format.
df_hmc_e["Ecode"] = df_hmc_e["Ecode"].apply(lambda x: "E" + re.sub(r'\W+', '', str(x)))
Ecodes = df_hmc_e["Ecode"].unique()
print(Ecodes.shape)
print(Ecodes)

(740,)
['E8000' 'E8001' 'E8002' 'E8003' 'E8008' 'E8009' 'E8010' 'E8011' 'E8012'
 'E8013' 'E8018' 'E8019' 'E8020' 'E8021' 'E8022' 'E8023' 'E8028' 'E8029'
 'E8030' 'E8031' 'E8032' 'E8033' 'E8038' 'E8039' 'E8040' 'E8041' 'E8042'
 'E8043' 'E8048' 'E8049' 'E8050' 'E8051' 'E8052' 'E8053' 'E8058' 'E8059'
 'E8060' 'E8061' 'E8062' 'E8063' 'E8068' 'E8069' 'E8070' 'E8071' 'E8072'
 'E8073' 'E8078' 'E8079' 'E8100' 'E8101' 'E8102' 'E8103' 'E8104' 'E8105'
 'E8106' 'E8107' 'E8108' 'E8109' 'E8110' 'E8111' 'E8112' 'E8113' 'E8114'
 'E8115' 'E8116' 'E8117' 'E8118' 'E8119' 'E8120' 'E8121' 'E8122' 'E8123'
 'E8124' 'E8125' 'E8126' 'E8127' 'E8128' 'E8129' 'E8130' 'E8131' 'E8132'
 'E8133' 'E8134' 'E8135' 'E8136' 'E8137' 'E8138' 'E8139' 'E8140' 'E8141'
 'E8142' 'E8143' 'E8144' 'E8145' 'E8146' 'E8147' 'E8148' 'E8149' 'E8150'
 'E8151' 'E8152' 'E8153' 'E8154' 'E8155' 'E8156' 'E8157' 'E8158' 'E8159'
 'E8160' 'E8161' 'E8162' 'E8163' 'E8164' 'E8165' 'E8166' 'E8167' 'E8168'
 'E8169' 'E8170' 'E8171' 'E8172' 'E8173' 'E8

In [5]:
def select_ICDcode_df(dataset_path):
	'''
	Select Data according to E-code of ICD9_code
	'''
	# HMC's list of traumatic injury E-codes. Detail
  # Detail saved at "ReferenceFiles/ICD_Nonpoisoning_Cause_Matrix.xlsx"
	Ecodes = np.array(
    ['E8000', 'E8001', 'E8002', 'E8003', 'E8008', 'E8009', 'E8010',
       'E8011', 'E8012', 'E8013', 'E8018', 'E8019', 'E8020', 'E8021',
       'E8022', 'E8023', 'E8028', 'E8029', 'E8030', 'E8031', 'E8032',
       'E8033', 'E8038', 'E8039', 'E8040', 'E8041', 'E8042', 'E8043',
       'E8048', 'E8049', 'E8050', 'E8051', 'E8052', 'E8053', 'E8058',
       'E8059', 'E8060', 'E8061', 'E8062', 'E8063', 'E8068', 'E8069',
       'E8070', 'E8071', 'E8072', 'E8073', 'E8078', 'E8079', 'E8100',
       'E8101', 'E8102', 'E8103', 'E8104', 'E8105', 'E8106', 'E8107',
       'E8108', 'E8109', 'E8110', 'E8111', 'E8112', 'E8113', 'E8114',
       'E8115', 'E8116', 'E8117', 'E8118', 'E8119', 'E8120', 'E8121',
       'E8122', 'E8123', 'E8124', 'E8125', 'E8126', 'E8127', 'E8128',
       'E8129', 'E8130', 'E8131', 'E8132', 'E8133', 'E8134', 'E8135',
       'E8136', 'E8137', 'E8138', 'E8139', 'E8140', 'E8141', 'E8142',
       'E8143', 'E8144', 'E8145', 'E8146', 'E8147', 'E8148', 'E8149',
       'E8150', 'E8151', 'E8152', 'E8153', 'E8154', 'E8155', 'E8156',
       'E8157', 'E8158', 'E8159', 'E8160', 'E8161', 'E8162', 'E8163',
       'E8164', 'E8165', 'E8166', 'E8167', 'E8168', 'E8169', 'E8170',
       'E8171', 'E8172', 'E8173', 'E8174', 'E8175', 'E8176', 'E8177',
       'E8178', 'E8179', 'E8180', 'E8181', 'E8182', 'E8183', 'E8184',
       'E8185', 'E8186', 'E8187', 'E8188', 'E8189', 'E8190', 'E8191',
       'E8192', 'E8193', 'E8194', 'E8195', 'E8196', 'E8197', 'E8198',
       'E8199', 'E8200', 'E8201', 'E8202', 'E8203', 'E8204', 'E8205',
       'E8206', 'E8207', 'E8208', 'E8209', 'E8210', 'E8211', 'E8212',
       'E8213', 'E8214', 'E8215', 'E8216', 'E8217', 'E8218', 'E8219',
       'E8220', 'E8221', 'E8222', 'E8223', 'E8224', 'E8225', 'E8226',
       'E8227', 'E8228', 'E8229', 'E8230', 'E8231', 'E8232', 'E8233',
       'E8234', 'E8235', 'E8236', 'E8237', 'E8238', 'E8239', 'E8240',
       'E8241', 'E8242', 'E8243', 'E8244', 'E8245', 'E8246', 'E8247',
       'E8248', 'E8249', 'E8250', 'E8251', 'E8252', 'E8253', 'E8254',
       'E8255', 'E8256', 'E8257', 'E8258', 'E8259', 'E8260', 'E8261',
       'E8262', 'E8263', 'E8264', 'E8268', 'E8269', 'E8270', 'E8272',
       'E8273', 'E8274', 'E8278', 'E8279', 'E8280', 'E8282', 'E8284',
       'E8288', 'E8289', 'E8290', 'E8294', 'E8298', 'E8299', 'E8300',
       'E8301', 'E8302', 'E8303', 'E8304', 'E8305', 'E8306', 'E8307',
       'E8308', 'E8309', 'E8310', 'E8311', 'E8312', 'E8313', 'E8314',
       'E8315', 'E8316', 'E8317', 'E8318', 'E8319', 'E8320', 'E8321',
       'E8322', 'E8323', 'E8324', 'E8325', 'E8326', 'E8327', 'E8328',
       'E8329', 'E8330', 'E8331', 'E8332', 'E8333', 'E8334', 'E8335',
       'E8336', 'E8337', 'E8338', 'E8339', 'E8340', 'E8341', 'E8342',
       'E8343', 'E8344', 'E8345', 'E8346', 'E8347', 'E8348', 'E8349',
       'E8350', 'E8351', 'E8352', 'E8353', 'E8354', 'E8355', 'E8356',
       'E8357', 'E8358', 'E8359', 'E8360', 'E8361', 'E8362', 'E8363',
       'E8364', 'E8365', 'E8366', 'E8367', 'E8368', 'E8369', 'E8370',
       'E8371', 'E8372', 'E8373', 'E8374', 'E8375', 'E8376', 'E8377',
       'E8378', 'E8379', 'E8380', 'E8381', 'E8382', 'E8383', 'E8384',
       'E8385', 'E8386', 'E8387', 'E8388', 'E8389', 'E8400', 'E8401',
       'E8402', 'E8403', 'E8404', 'E8405', 'E8406', 'E8407', 'E8408',
       'E8409', 'E8410', 'E8411', 'E8412', 'E8413', 'E8414', 'E8415',
       'E8416', 'E8417', 'E8418', 'E8419', 'E8426', 'E8427', 'E8428',
       'E8429', 'E8430', 'E8431', 'E8432', 'E8433', 'E8434', 'E8435',
       'E8436', 'E8437', 'E8438', 'E8439', 'E8440', 'E8441', 'E8442',
       'E8443', 'E8444', 'E8445', 'E8446', 'E8447', 'E8448', 'E8449',
       'E8450', 'E8458', 'E8459', 'E8460', 'E8470', 'E8480', 'E8800',
       'E8801', 'E8809', 'E8810', 'E8811', 'E8820', 'E8830', 'E8831',
       'E8832', 'E8839', 'E8840', 'E8841', 'E8842', 'E8843', 'E8844',
       'E8845', 'E8846', 'E8849', 'E8850', 'E8851', 'E8852', 'E8853',
       'E8854', 'E8859', 'E8860', 'E8869', 'E8870', 'E8880', 'E8881',
       'E8888', 'E8889', 'E8900', 'E8901', 'E8902', 'E8903', 'E8908',
       'E8909', 'E8910', 'E8911', 'E8912', 'E8913', 'E8918', 'E8919',
       'E8920', 'E8930', 'E8931', 'E8932', 'E8938', 'E8939', 'E8940',
       'E8950', 'E8960', 'E8970', 'E8980', 'E8981', 'E8990', 'E9000',
       'E9001', 'E9009', 'E9010', 'E9011', 'E9018', 'E9019', 'E9020',
       'E9021', 'E9022', 'E9028', 'E9029', 'E9030', 'E9040', 'E9041',
       'E9042', 'E9043', 'E9049', 'E9050', 'E9051', 'E9052', 'E9053',
       'E9054', 'E9055', 'E9056', 'E9057', 'E9060', 'E9061', 'E9062',
       'E9063', 'E9064', 'E9065', 'E9068', 'E9069', 'E9070', 'E9080',
       'E9081', 'E9082', 'E9083', 'E9084', 'E9088', 'E9089', 'E9090',
       'E9091', 'E9092', 'E9093', 'E9094', 'E9098', 'E9099', 'E9100',
       'E9101', 'E9102', 'E9103', 'E9104', 'E9108', 'E9109', 'E9110',
       'E9120', 'E9130', 'E9131', 'E9132', 'E9133', 'E9138', 'E9139',
       'E9140', 'E9150', 'E9160', 'E9170', 'E9171', 'E9172', 'E9173',
       'E9174', 'E9175', 'E9176', 'E9177', 'E9178', 'E9179', 'E9180',
       'E9190', 'E9191', 'E9192', 'E9193', 'E9194', 'E9195', 'E9196',
       'E9197', 'E9198', 'E9199', 'E9200', 'E9201', 'E9202', 'E9203',
       'E9204', 'E9205', 'E9208', 'E9209', 'E9210', 'E9211', 'E9218',
       'E9219', 'E9220', 'E9221', 'E9222', 'E9223', 'E9224', 'E9225',
       'E9228', 'E9229', 'E9230', 'E9231', 'E9232', 'E9238', 'E9239',
       'E9240', 'E9241', 'E9242', 'E9248', 'E9249', 'E9250', 'E9251',
       'E9252', 'E9258', 'E9259', 'E9270', 'E9271', 'E9272', 'E9273',
       'E9274', 'E9278', 'E9279', 'E9280', 'E9281', 'E9282', 'E9283',
       'E9284', 'E9285', 'E9286', 'E9287', 'E9288', 'E9289', 'E9507',
       'E9520', 'E9521', 'E9530', 'E9531', 'E9538', 'E9539', 'E9540',
       'E9550', 'E9551', 'E9552', 'E9553', 'E9554', 'E9555', 'E9556',
       'E9557', 'E9559', 'E9560', 'E9570', 'E9571', 'E9572', 'E9579',
       'E9580', 'E9581', 'E9582', 'E9583', 'E9584', 'E9585', 'E9586',
       'E9587', 'E9588', 'E9589', 'E9600', 'E9610', 'E9620', 'E9621',
       'E9622', 'E9630', 'E9640', 'E9650', 'E9651', 'E9652', 'E9653',
       'E9654', 'E9655', 'E9656', 'E9657', 'E9658', 'E9659', 'E9660',
       'E9670', 'E9671', 'E9672', 'E9673', 'E9674', 'E9675', 'E9676',
       'E9677', 'E9678', 'E9679', 'E9680', 'E9681', 'E9682', 'E9683',
       'E9684', 'E9685', 'E9686', 'E9687', 'E9688', 'E9689', 'E9700',
       'E9710', 'E9720', 'E9730', 'E9740', 'E9750', 'E9760', 'E9780',
       'E9790', 'E9791', 'E9792', 'E9793', 'E9794', 'E9795', 'E9796',
       'E9797', 'E9798', 'E9799', 'E9806', 'E9830', 'E9831', 'E9838',
       'E9839', 'E9840', 'E9850', 'E9851', 'E9852', 'E9853', 'E9854',
       'E9855', 'E9856', 'E9857', 'E9860', 'E9870', 'E9871', 'E9872',
       'E9879', 'E9880', 'E9881', 'E9882', 'E9883', 'E9884', 'E9885',
       'E9886', 'E9888', 'E9889', 'E9900', 'E9901', 'E9902', 'E9903',
       'E9909', 'E9910', 'E9911', 'E9912', 'E9913', 'E9914', 'E9915',
       'E9916', 'E9917', 'E9918', 'E9919', 'E9920', 'E9921', 'E9922',
       'E9923', 'E9928', 'E9929', 'E9930', 'E9931', 'E9932', 'E9933',
       'E9934', 'E9935', 'E9936', 'E9937', 'E9938', 'E9939', 'E9940',
       'E9941', 'E9942', 'E9943', 'E9948', 'E9949', 'E9950', 'E9951',
       'E9952', 'E9953', 'E9954', 'E9958', 'E9959', 'E9960', 'E9961',
       'E9968', 'E9969', 'E9970', 'E9971', 'E9972', 'E9973', 'E9978',
       'E9979', 'E9980', 'E9981', 'E9988', 'E9989'], dtype=object)


	# Load the diagnoses table.
	file_path = os.path.join(dataset_path, 'DIAGNOSES_ICD.csv.gz')
	ICDDIAGNOSES_df = utils.load_df(file_path)
	# Drop rows with NaN values in column 'ICD9_CODE'
	ICDDIAGNOSES_df = ICDDIAGNOSES_df.dropna(subset=['ICD9_CODE'])

	# select "ICD9_CODE" starting with E
	trum_df = ICDDIAGNOSES_df[ICDDIAGNOSES_df['ICD9_CODE'].str.startswith('E')]
	trum_df.loc[:, "ICD9_CODE"] = trum_df.loc[:, "ICD9_CODE"].apply(lambda e: str(e)+"0" if len(str(e)) < 5 else e)

	# Select all E-code diagnoses.
	trum_df = trum_df[trum_df['ICD9_CODE'].isin(Ecodes)]

	return trum_df
# df = select_ICDcode_df(DATASET_PATH)
# print(df.shape)
# df.nunique()

In [ ]:
file_path = os.path.join(DATASET_PATH, 'DIAGNOSES_ICD.csv.gz')
ICDDIAGNOSES_df = utils.load_df(file_path)

In [ ]:
ICDDIAGNOSES_df.ICD9_CODE.unique()
ICDDIAGNOSES_df = ICDDIAGNOSES_df.dropna(subset=['ICD9_CODE'])
ICDDIAGNOSES_df = ICDDIAGNOSES_df[ICDDIAGNOSES_df['ICD9_CODE'].str.startswith('E')]
ICDDIAGNOSES_df.loc[:, "ICD9_CODE"] = ICDDIAGNOSES_df.loc[:, "ICD9_CODE"].apply(lambda e: str(e)+"0" if len(str(e)) < 5 else e)

In [ ]:
ICDDIAGNOSES_df.shape, ICDDIAGNOSES_df.ICD9_CODE.nunique(),
# ICDDIAGNOSES_df.sort_values('ICD9_CODE').ICD9_CODE.unique()

((22544, 4), 500)

In [ ]:
trum_df = ICDDIAGNOSES_df[ICDDIAGNOSES_df['ICD9_CODE'].isin(Ecodes)]
trum_df.shape, trum_df.ICD9_CODE.nunique()
# trum_df.sort_values('ICD9_CODE').ICD9_CODE.unique()

((6423, 4), 236)

In [ ]:
not_trum_df = ICDDIAGNOSES_df[~(ICDDIAGNOSES_df['ICD9_CODE'].isin(Ecodes))]
not_trum_df.shape, not_trum_df.ICD9_CODE.nunique()
not_trum_df.sort_values('ICD9_CODE').ICD9_CODE.unique()

array(['E0000', 'E0008', 'E0009', 'E0010', 'E0011', 'E0020', 'E0026',
       'E0030', 'E0031', 'E0032', 'E0039', 'E0060', 'E0061', 'E0062',
       'E0064', 'E0069', 'E0070', 'E0071', 'E0073', 'E0076', 'E0080',
       'E0089', 'E0138', 'E0139', 'E0161', 'E0162', 'E0190', 'E0291',
       'E0299', 'E0300', 'E8490', 'E8493', 'E8494', 'E8495', 'E8496',
       'E8497', 'E8498', 'E8499', 'E8500', 'E8501', 'E8502', 'E8503',
       'E8504', 'E8508', 'E8510', 'E8528', 'E8529', 'E8532', 'E8538',
       'E8540', 'E8541', 'E8542', 'E8543', 'E8548', 'E8550', 'E8551',
       'E8552', 'E8554', 'E8555', 'E8556', 'E8560', 'E8570', 'E8580',
       'E8581', 'E8582', 'E8583', 'E8584', 'E8585', 'E8586', 'E8587',
       'E8588', 'E8589', 'E8600', 'E8603', 'E8609', 'E8611', 'E8613',
       'E8619', 'E8624', 'E8637', 'E8641', 'E8654', 'E8655', 'E8663',
       'E8668', 'E8669', 'E8694', 'E8698', 'E8700', 'E8702', 'E8703',
       'E8704', 'E8705', 'E8706', 'E8708', 'E8709', 'E8710', 'E8714',
       'E8716', 'E87

# Demographics
------------------------------------------------------------------
Title: Detailed information on ICUSTAY_ID
Description: This query provides a useful set of information regarding patient
             ICU stays. The information is combined from the admissions,patients, and icustays tables. It includes age, length of stay, sequence, and expiry flags.
MIMIC version: MIMIC-III v1.4
A modify veristion of: https://github.com/MIT-LCP/mimic-code/blob/main/mimic-iii/concepts/demographics/icustay_detail.sql
modified list:
  1. duration result is expressed in hours instead of in days
------------------------------------------------------------------
this table includes:
*   Patient Info:
 >  'subject_id', 'gender', 'dod', 'ethnicity','ethnicity_grouped',
*   Hosipital Admission Info:
 >  'hadm_id', 'admittime', 'dischtime', 'los_hospital_hours', 'admission_age' 'hospital_expire_flag'(0/1), 'hospstay_seq', 'first_hosp_stay'(T/F)
* ICUstay Info:
 > 'icustay_id', 'intime', 'outtime', 'los_icu_hours', 'icustay_seq', 'first_icu_stay'(T/F)



In [6]:
# Check if the file exists
demog_path = os.path.join(PREPROCESS_DATA_PATH, "demographics.csv")
if os.path.exists(demog_path):
    # Load the CSV file into a DataFrame
    demog_df = pd.read_csv(demog_path, index_col=0)
else:
    # Query demographics information by using BigQuery
    demog_df = utils.demog_sql2df(PROJECT_ID, saved_path=PREPROCESS_DATA_PATH)
# demog_df = demog_df[['subject_id', 'hadm_id', 'icustay_id','admission_age', 'los_hospital_hours']]

# Ventilation

Based on sorece file: [ventilation_classification.sql](https://github.com/MIT-LCP/mimic-code/blob/main/mimic-iii/concepts/durations/ventilation_classification.sql)


Detailes saved at:ReferenceFiles/[Ventilation_Evaluation.ipynb](https://colab.research.google.com/drive/1e_75eW1D3HZ2dECoJQ6iqAzCQrq90zeI#scrollTo=ogGVrDQghSj2)


In [7]:
import mimiciii_icu_trauma_patient_cohort as trauma_cohort
# get ventilation day table
vent_path = os.path.join(PREPROCESS_DATA_PATH, "ventilation_day_MV.csv")
if os.path.exists(vent_path):
    vent_df = pd.read_csv(vent_path, index_col=0)
else:
    vent_df = trauma_cohort.ventilation_day_count(PROJECT_ID, vent_type=['MechVent'])
    vent_df.to_csv(vent_path)
# vent_df.shape

In [ ]:
# If we count all 4 type of ventilation
# vent_day_count = ventilation_day_count(PROJECT_ID, vent_type=['MechVent','OxygenTherapy','Extubated','SelfExtubated'])
# # If we only count critical feature/variable: MechVent (i.e., mechanical ventilation)
# vent_day_count = ventilation_day_count(PROJECT_ID, vent_type=['MechVent'])

# Summary

In [15]:
from mimiciii_icu_trauma_patient_cohort import ventilation_day_count

def trum_population_ids(dataset_path, preprocess_data_path,
                        project_id,
                        vent_type=['MechVent'], vent_threshold=None,
                        is_report=False):
  """
  Trauma Population Selection Process
    1. Select Data according to E-code of ICD9_code
    2. Age between [18, 89]
    3. ADM_duration between [48, None)
    4. Ventilation days >= 3
       -> this is according to vent_type and
       vent_type:
          -> By default we only select critical feature MechVent(i.e., mechanical ventilation)
          -> OR we can also select all 4 types: ['MechVent','OxygenTherapy','Extubated','SelfExtubated']
          -> though the reasonability of including 'OxygenTherapy' and 'Extubated' is questionable
       vent_threshold:
          -> default is None, not includes MV filter
          -> or a int, the lower lound for MV filter.
          -> this bounderay is experimental
          -> in Stern's paper, they use 3 as threshold
            ( Stern K, Qiu Q, Weykamp M, O’Keefe G, Brakenridge SC.
              Defining Posttraumatic Sepsis for Population-Level Research.
              JAMA Netw Open. 2023;6(1):e2251445. doi:10.1001/jamanetworkopen.2022.51445 )
  """
  # Get all pacients
  # Check if the file exists
  demog_path = os.path.join(preprocess_data_path, "demographics.csv")
  if os.path.exists(demog_path):
      # Load the CSV file into a DataFrame
      demog_df = pd.read_csv(demog_path, index_col=0)
  else:
      # Query demographics information by using BigQuery
      demog_df = utils.demog_sql2df(project_id, saved_path=preprocess_data_path)
  demog_df = demog_df[['subject_id', 'hadm_id', 'icustay_id','admission_age', 'los_hospital_hours', 'los_hospital_days', 'hospital_expire_flag']]
  count_df = demog_df[['subject_id', 'hadm_id', 'icustay_id']].nunique().to_frame(name='TOTAL')
  print("MIMIC III includes %d hospital admission"%demog_df.hadm_id.nunique())

  # Selected accordind to E-code,
  # group by IDs and agg ICD9_CODE info, because we want unique IDs
  TRUM_df = select_ICDcode_df(dataset_path).groupby(['SUBJECT_ID', 'HADM_ID'])['ICD9_CODE'].agg(set).reset_index()
  TRUM_df = demog_df[demog_df['hadm_id'].isin(TRUM_df.HADM_ID)]
  print("After Truama selection (ICD-9): ", TRUM_df.hadm_id.nunique())
  # filter according to the age in range [18, 89]
  TRUM_df = utils.filter_col_value(TRUM_df, 'admission_age', [18,89])
  # count_df['TRUM basic filter'] = TRUM_df[['subject_id', 'hadm_id', 'icustay_id']].nunique()
  print("After Age Filter: ", TRUM_df.hadm_id.nunique())

  # filter according to the HADM duration in range [48h, NO upper bound)
  TRUM_df_firstfewday = TRUM_df.loc[TRUM_df.los_hospital_hours<48, ['hadm_id', 'hospital_expire_flag']].drop_duplicates()
  TRUM_df_firstfewday_mortalitycount = TRUM_df_firstfewday.hospital_expire_flag.value_counts()
  TRUM_df = utils.filter_col_value(TRUM_df, 'los_hospital_hours', [48, None])
  print(f"After Hospital Length of Stay >= 48h Filter: %d"%(TRUM_df.hadm_id.nunique()))
  # TRUM_df = utils.filter_col_value(TRUM_df, 'los_hospital_hours', [2, None])
  # print(f"After Hospital Length of Stay >= 2d filter: %d"%(TRUM_df.hadm_id.nunique()))
  print(f"\t Hospital Length of Stay  < 48h: %d = %d(Died) + %d(Discharged Alive)"%(TRUM_df_firstfewday.shape[0],TRUM_df_firstfewday_mortalitycount[1], TRUM_df_firstfewday_mortalitycount[0]))
  count_df['TRUM basic filter'] = TRUM_df[['subject_id', 'hadm_id', 'icustay_id']].nunique()


  # Select accoring Ventilation Days
  # get ventilation day table
  if vent_type == ["MechVent"]:
    vent_path = os.path.join(preprocess_data_path, "ventilation_day_MV.csv")
    if os.path.exists(vent_path):
        vent_day_count = pd.read_csv(vent_path, index_col=0)
    else:
        vent_day_count = ventilation_day_count(PROJECT_ID, vent_type=vent_type)
        vent_day_count.to_csv(vent_path)
  elif vent_type == ['MechVent','OxygenTherapy','Extubated','SelfExtubated']:
    vent_path = os.path.join(preprocess_data_path, "ventilation_day_all.csv")
    if os.path.exists(vent_path):
        vent_day_count = pd.read_csv(vent_path, index_col=0)
    else:
        vent_day_count = ventilation_day_count(PROJECT_ID, vent_type=vent_type)
        vent_day_count.to_csv(vent_path)
  else:
    print("WARNING: unusual ventilation type list ")
    vent_day_count = ventilation_day_count(PROJECT_ID, vent_type=vent_type)
  #select accoring Ventilation days >= vent_threshold(default is 3) for each pacients(HADM_id)
  TRUM_df = TRUM_df.merge(vent_day_count, on='hadm_id', how='left')
  trum_vent_day_count = TRUM_df[['hadm_id','date_count']].drop_duplicates()
  num_NotIntubated = trum_vent_day_count.date_count.isna().sum()
  Intubated_less_thr = trum_vent_day_count[trum_vent_day_count.date_count<vent_threshold].shape[0]
  if vent_threshold!=None:
    TRUM_df = TRUM_df[TRUM_df['date_count'] >= vent_threshold]
    count_df['TRUM Vent filter'] = TRUM_df[['subject_id', 'hadm_id', 'icustay_id']].nunique()
    print("Final Cohort Size:", TRUM_df.hadm_id.nunique())
    print('\tMV fillter out: %d(Not Intubated) + %d(Intubated < %d)'%(num_NotIntubated, Intubated_less_thr, vent_threshold))

  # Statistics ###
  if is_report:
    display(count_df)
  return TRUM_df

TRUM_df = trum_population_ids(DATASET_PATH, PREPROCESS_DATA_PATH, PROJECT_ID,
                              vent_threshold=2,
                              is_report=True)

MIMIC III includes 57328 hospital admission
After Truama selection (ICD-9):  6148
After Age Filter:  5651
After Hospital Length of Stay >= 48h Filter: 4977
	 Hospital Length of Stay  < 48h: 674 = 200(Died) + 474(Discharged Alive)
Final Cohort Size: 2242
	MV fillter out: 2271(Not Intubated) + 464(Intubated < 2)


,TOTAL,TRUM basic filter,TRUM Vent filter
subject_id,46428,4826,2209
hadm_id,57328,4977,2242
icustay_id,61051,5410,2545


In [16]:
!ls

 1.4					 preprocess			   trauma_ids_0502
 mimiciii_icu_trauma_patient_cohort.py	 spesis_assignment_preprocess.py   Trum_cohort_character
 mimic_utils.py				 Supplementary
 Notebooks				'Survey Notebooks'


In [17]:
TRUM_df.to_csv(os.path.join("trauma_ids"))

# Other Config

In [11]:
# # with out MV day >=3 day filter
# TRUM_df = trum_population_ids(DATASET_PATH, PREPROCESS_DATA_PATH, PROJECT_ID,
#                               vent_threshold=3,
#                               is_report=True)

MIMIC III includes 57328 hospital admission
After Truama selection (ICD-9):  6148
After Age Filter:  5651
After Hospital Length of Stay >= 48h Filter: 4977
	 Hospital Length of Stay  < 48h: 674 = 200(Died) + 474(Discharged Alive)
Final Cohort Size: 1570
	MV fillter out: 2271(Not Intubated) + 1136(Intubated < 3)


,TOTAL,TRUM basic filter,TRUM Vent filter
subject_id,46428,4826,1551
hadm_id,57328,4977,1570
icustay_id,61051,5410,1828


In [13]:
# # with out MV day filter
# TRUM_df = trum_population_ids(DATASET_PATH, PREPROCESS_DATA_PATH, PROJECT_ID,
#                     vent_type=['MechVent'], vent_threshold=None,
#                     is_report=True)

MIMIC III includes 57328 hospital admission
After Truama selection (ICD-9):  6148
After Age Filter:  5651
After Hospital Length of Stay >= 48h Filter: 4977
	 Hospital Length of Stay  < 48h: 674 = 200(Died) + 474(Discharged Alive)


,TOTAL,TRUM basic filter
subject_id,46428,4826
hadm_id,57328,4977
icustay_id,61051,5410


In [14]:
# # If we count all 4 type of ventilation
# TRUM_df = trum_population_ids(DATASET_PATH, PREPROCESS_DATA_PATH, PROJECT_ID,
#                               vent_type=['MechVent','OxygenTherapy','Extubated','SelfExtubated'], vent_threshold=3, is_report=True)

MIMIC III includes 57328 hospital admission
After Truama selection (ICD-9):  6148
After Age Filter:  5651
After Hospital Length of Stay >= 48h Filter: 4977
	 Hospital Length of Stay  < 48h: 674 = 200(Died) + 474(Discharged Alive)
Final Cohort Size: 2742
	MV fillter out: 506(Not Intubated) + 1729(Intubated < 3)


,TOTAL,TRUM basic filter,TRUM Vent filter
subject_id,46428,4826,2693
hadm_id,57328,4977,2742
icustay_id,61051,5410,3145


# first care unit location
For Study Population in Katie's paper
> We excluded individuals whose **first inpatient hospital unit** was a location **other than the ICU or an emergency procedural unit** (eg, operating room or angiography)


Source file **TRANSFERS** Table
> Physical locations for patients throughout their hospital stay.

In [ ]:
# load transfers table
# /content/drive/MyDrive/Yin_AutoEncoder/mimiciii_code/Project/1.4/TRANSFERS.csv.gz
TRANSFERS = utils.load_df(os.path.join(DATASET_PATH, 'TRANSFERS.csv.gz'))
TRANSFERS.rename(str.lower, axis=1, inplace=True)
trum_location = TRUM_df[['hadm_id']].drop_duplicates().merge(TRANSFERS, on='hadm_id')
admit_location_df = trum_location[trum_location.eventtype=='admit']#.curr_careunit.unique()
# print(admit_location_df.shape)
first_location_df = trum_location[trum_location.curr_careunit.notna()].sort_values(['hadm_id', 'intime']).groupby('hadm_id').agg(lambda x:x.head(1)).reset_index()
# print(first_location_df.shape)
first_location_df.groupby('curr_careunit')[['hadm_id']].count()

(1570, 12)
(1570, 12)


,hadm_id
curr_careunit,
CCU,74
CSRU,93
MICU,282
SICU,203
TSICU,918


In [ ]:
display(admit_location_df.groupby('curr_careunit')[['hadm_id']].count())
admit_location_df.curr_careunit.unique()

,hadm_id
curr_careunit,
CCU,48
CSRU,62
MICU,179
SICU,142
TSICU,803


array(['TSICU', 'MICU', nan, 'CSRU', 'SICU', 'CCU'], dtype=object)

In [ ]:
# admit to curr_careunit=NaN
TRANSFERS.loc[TRANSFERS.hadm_id==100132,
              ['hadm_id', 'icustay_id',
               'eventtype', 'prev_careunit', 'curr_careunit', 'prev_wardid', 'curr_wardid',
               'intime', 'outtime', 'los']]#.curr_careunit.unique()

,hadm_id,icustay_id,eventtype,prev_careunit,curr_careunit,prev_wardid,curr_wardid,intime,outtime,los
ROW_ID,,,,,,,,,,
108312,100132,NaN,admit,NaN,NaN,NaN,4.0,2179-03-05 16:37:12,2179-03-05 17:10:33,0.56
108313,100132,NaN,transfer,NaN,NaN,4.0,27.0,2179-03-05 17:10:33,2179-03-05 20:14:43,3.07
108314,100132,229201.0,transfer,NaN,MICU,27.0,52.0,2179-03-05 20:14:43,2179-03-24 11:37:33,447.38
108315,100132,NaN,discharge,MICU,NaN,52.0,NaN,2179-03-24 11:37:33,NaN,NaN


In [ ]:
first_location_df[['hadm_id', 'intime', 'eventtype', 'prev_careunit','curr_careunit']].sort_values(['hadm_id', 'intime'])

,hadm_id,intime,eventtype,prev_careunit,curr_careunit
6413,100011,2177-08-29 04:52:21,admit,NaN,TSICU
4476,100035,2115-02-22 06:52:06,admit,NaN,MICU
2460,100132,2179-03-05 16:37:12,admit,NaN,NaN
5669,100133,2196-09-27 04:47:25,admit,NaN,NaN
4135,100138,2179-09-22 00:21:42,admit,NaN,CCU
...,...,...,...,...,...
2008,199877,2195-10-08 22:53:15,admit,NaN,SICU
5772,199909,2174-02-20 13:12:11,admit,NaN,TSICU
703,199931,2199-07-10 21:47:06,admit,NaN,SICU
4713,199962,2118-07-16 05:42:45,admit,NaN,TSICU
